In [1]:
from PIL import Image
import numpy as np
import random
import copy
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import glob

### 1st Step: Obtain the forest image and the 'burnt region' used to simulate the burnt area

In [2]:
def convert_to_grayscale(image):
    grayscale = image.convert("L")
    return grayscale


def obtain_img_burnt(image_name, burnt_name):
    img = Image.open(image_name).convert("RGB")
    burnt = Image.open(burnt_name)
    burnt_array = np.array(burnt)
    grayscale = convert_to_grayscale(img)
    array_grayscale = np.stack((grayscale,)*3, axis=-1)
    return img, array_grayscale, burnt

### 2nd Step: All the image put as grayscale and put some pixels fo burnt area  

In [3]:
def burnt_all(array_image, array_grayscale, burnt_array):
    new_image = copy.deepcopy(array_grayscale)
    for i in range(0, array_image.shape[0]):
        for j in range(0, array_image.shape[1]):
            #for rgb in range(0, 1):
            new_image[i][j][0] = burnt_array[i%burnt_array.shape[0]][j%burnt_array.shape[1]][0] 
            new_image[i][j][2] = burnt_array[i%burnt_array.shape[0]][j%burnt_array.shape[1]][2]                
            if random.random()<0.15:
                new_image[i][j][1] = burnt_array[i%burnt_array.shape[0]][j%burnt_array.shape[1]][1]                

    img_modified = Image.fromarray(new_image)
    gray_img_modified = np.stack((convert_to_grayscale(img_modified),)*3, axis=-1)
    for i in range(0, gray_img_modified.shape[0]):
        for j in range(0, gray_img_modified.shape[1]):
            gray_img_modified[i][j][2] = gray_img_modified[i][j][2] + 3
    return gray_img_modified

### 3rd Step: Obtain which region will be burnt area:

In [4]:
def burnt_region(region_array):
    for i in range(0, region_array.shape[0]):
        for j in range(0, region_array.shape[1]):
            if not np.array_equal(region_array[i][j],np.array([0, 0, 0])):
                region_array[i][j] = np.array([255, 0, 255 ])
    return region_array

### 4th Step: Finally put the burnt region correctly and fill with the correct colour.

In [5]:
def add_burnt_region(img, all_burnt_array, region_array, alpha, position):
    img_array = np.array(img)
    background = Image.fromarray(img_array)
    background.paste(Image.fromarray(region_array), position)
    background = background.convert("RGB")
    background_array = np.array(background)
    for i in range(0, background_array.shape[0]):
        for j in range(0, background_array.shape[1]):
            if np.array_equal(background_array[i][j], np.array([240, 0, 0])):
                background_array[i][j] = all_burnt_array[i][j] 
            else:
                background_array[i][j] = img_array[i][j]
    background = Image.fromarray(background_array)
    return background

### 5th Step: Finally put the burnt region correctly and fill with the correct colour.

In [6]:
def generate_segmented_image(img, region_array, position, image_name):
    n, m, rgb = np.array(img).shape
    image = Image.new('RGB', (m, n))
    region = Image.fromarray(region_array)
    image.paste(region, position)
    print("> Generated segmented image")
    image.save(image_name.replace("images_located", "images_located_segmented").replace(".jpg",".png"))

### 6th Step: Final pipe line

In [7]:
def generate_image(image_name, burnt_name, region_name, alpha, position):
    print("> Starting to process image: '%s' with burnt region '%s' and region '%s'." % ((image_name, burnt_name, region_name)))
    img, array_grayscale, burnt = obtain_img_burnt(image_name, burnt_name)
    print("> 1st step done.")
    all_burnt_array = burnt_all(np.array(img), array_grayscale, np.array(burnt))
    print("> 2nd step done.")
    region_array = np.array(Image.open(region_name))
    print("> 3rd step done.")
    generate_segmented_image(img, region_array, position, image_name)
    result_image = add_burnt_region(img, all_burnt_array, region_array, alpha, position)
    print("> 4th step done.")
    name = green.replace(".png", "_%s_.png"% region.replace("regions/","").replace(".png", ""))
    result_image.save(image_name.replace("images_located", "images_located_burnt"))
    print("-------- SAVED --------")

### Image generation

In [8]:
region_files = ['regions/region16.png', 'regions/region26.png', 'regions/region58.png', 'regions/region37.png']

In [10]:
green_files = glob.glob("images_located/Cap*.png")
burnt_name = "burnt.png"
metadata = {'id_green':[], 'id_region':[], 'position':[], 'id_result':[]}
for green in green_files:
    w, h = Image.open(green).size
    position = (random.randint(10, round(w/3)), random.randint(10, round(h/3)))
    region_name = region_files[random.randint(0, len(region_files)-1)]
    metadata['id_green'] += [green]
    metadata['id_region'] += [region_name]
    metadata['position'] += [position]
    metadata['id_result'] += [green.replace("images_located", "images_located_burnt")]
    result_image = generate_image(green, burnt_name, region_name, 0.9, position)


> Starting to process image: 'images_located/Captura11.png' with burnt region 'burnt.png' and region 'regions/region58.png'.
> 1st step done.
> 2nd step done.
> 3rd step done.
> Generated segmented image
> 4th step done.
-------- SAVED --------
> Starting to process image: 'images_located/Captura10.png' with burnt region 'burnt.png' and region 'regions/region16.png'.
> 1st step done.
> 2nd step done.
> 3rd step done.
> Generated segmented image
> 4th step done.
-------- SAVED --------
> Starting to process image: 'images_located/Captura12.png' with burnt region 'burnt.png' and region 'regions/region16.png'.
> 1st step done.
> 2nd step done.
> 3rd step done.
> Generated segmented image
> 4th step done.
-------- SAVED --------
> Starting to process image: 'images_located/Captura13.png' with burnt region 'burnt.png' and region 'regions/region26.png'.
> 1st step done.
> 2nd step done.
> 3rd step done.
> Generated segmented image
> 4th step done.
-------- SAVED --------
> Starting to proces

In [11]:
pd.DataFrame(metadata).to_csv("metadata_images_located.csv")


In [31]:
import pandas as pd
import glob

image = "images_located/Captura17.png"
df_images_located = pd.read_csv("metadata_images_located.csv")
df_regions = pd.read_csv("regions.csv")
df_regions.head()

,Unnamed: 0,region_id,area
0,0,regions/region47.png,19.9491
1,1,regions/region53.png,14.4750
2,2,regions/region84.png,17.6589
3,3,regions/region90.png,23.3904
4,4,regions/region91.png,11.5563


In [32]:
region = df_images_located[df_images_located['id_green'] == image]['id_region'].values[0]
area = df_regions[df_regions['region_id'] == region]['area'].values[0]

In [34]:
region

'regions/region58.png'